In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

## Our role: 

Consultant for "Delta Air Lines Inc." 
We want to:
- Predict if a flight will be delayed or not at arrival
- Predict the delay time of a flight at arrival
- Find the most important possible recurrent reasons of delay

## To Do


- target variables: Cancelled (classification) and ArrDelayMinutes (regression)
- airtime: compute the average for each trip?
- ArrDelay: Compute the mean/median for each specific trip. Make sure that we have enough data points. 
- Drop the "Cancelled" rows

- Average delay per airline per flight per trip ---> score. Instead of Airline Name: avg airline delay in one column --> continuous variable.  If missing data --> 0.
- Check the different values Marketing_Airline_Network. PA1? PA2?
- Tail Number - One Aircraft. Compute the average delay of each particular aircraft. Instead of having the tail number, we have the average delay per "distances" of that aircraft.
- FlightNumber: Compute the average delay per flight number.
- OriginAirportID: Compute the average delay per airport.


In [3]:
df = pd.read_csv("../data/delta_combined_flights_2018_2022.csv")

In [4]:
df.head()

,FlightDate,Airline,Origin,Dest,DepTime,DepDelayMinutes,DepDelay,ArrDelayMinutes,AirTime,Distance,CRSElapsedTime,DayOfWeek,Year,Month,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,DestAirportID,DepDel15,DepartureDelayGroups,TaxiOut,CRSArrTime,ArrDelay,DistanceGroup
0,2018-01-27,Delta Air Lines Inc.,ATL,ORF,1606.0,0.0,-2.0,0.0,73.0,516.0,95.0,6,2018,1,N923DL,1378,10397,13931,0.0,-1.0,8.0,1743,-13.0,3
1,2018-01-27,Delta Air Lines Inc.,ATL,GSO,1656.0,0.0,-4.0,0.0,48.0,306.0,75.0,6,2018,1,N912DE,1380,10397,11995,0.0,-1.0,10.0,1815,-18.0,2
2,2018-01-27,Delta Air Lines Inc.,TPA,MSP,716.0,0.0,-4.0,0.0,173.0,1306.0,207.0,6,2018,1,N699DL,1381,15304,13487,0.0,-1.0,15.0,947,-19.0,6
3,2018-01-27,Delta Air Lines Inc.,LIH,LAX,2219.0,0.0,-6.0,7.0,311.0,2615.0,319.0,6,2018,1,N544US,1382,12982,12892,0.0,-1.0,17.0,544,7.0,11
4,2018-01-27,Delta Air Lines Inc.,SMF,SLC,553.0,0.0,-7.0,0.0,73.0,532.0,103.0,6,2018,1,N3766,1384,14893,14869,0.0,-1.0,12.0,843,-19.0,3
